# VGG16

In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Sequential
from glob import glob
import matplotlib.pyplot as plt

# from keras.optimizers import SGD, RMSprop
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
import cv2
import glob
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.python.keras import backend as K
import plotly.graph_objects as go
import plotly.offline as py
autosize =False

from plotly.subplots import make_subplots
import plotly.graph_objects as go

%matplotlib inline

In [ ]:
x_train = np.load('/kaggle/input/esophageal-endoscopy-images-resized/esophagus-cancer-files/x_train_128.npy')
x_train.shape

In [ ]:
y = pd.read_csv('/kaggle/input/esophageal-endoscopy-images-resized/esophagus-cancer-files/traindata.csv').target
y.shape

In [ ]:
from sklearn.utils import class_weight
 
class_weights = class_weight.compute_class_weight(class_weight="balanced",classes=np.unique(y),y=y)

In [ ]:
class_weights = dict(enumerate(class_weights))
class_weights

In [ ]:
#splitting the dataset
from sklearn.model_selection import train_test_split
train_imgs, validation_imgs, y_train, y_val = train_test_split(x_train,y, test_size=0.2, random_state=1234)
train_imgs, test_imgs, y_train, y_test = train_test_split(train_imgs,y_train, test_size=0.1, random_state=1234)

In [ ]:
test_imgs.shape

In [ ]:
batch_size = 128
num_classes = 2
epochs = 50
input_shape = (128,128,3)

In [ ]:
opt = Adam(lr=1e-5)

nb_train_steps = train_imgs.shape[0]//batch_size
nb_val_steps=validation_imgs.shape[0]//batch_size

print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

# no need to create augmentation images for validation data, only rescaling the pixels
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_imgs, y_train, batch_size=32)
val_generator = val_datagen.flow(validation_imgs, y_val, batch_size=32)

### Activating the trainable layers

In [ ]:
from keras.applications import vgg16
from keras.models import Model
import keras

vgg = vgg16.VGG16(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)

output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)
vgg_model = Model(vgg.input, output)

vgg_model.trainable = False
for layer in vgg_model.layers:
    layer.trainable = False
    
import pandas as pd
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])  

In [ ]:
vgg_model.trainable = True

set_trainable = False
for layer in vgg_model.layers:
    if layer.name in ['block5_conv1', 'block4_conv1']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

Adding layers for classification

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers

model = Sequential()
model.add(vgg_model)
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC()],optimizer=opt)

In [ ]:
model.summary()

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=nb_train_steps, epochs=epochs,
                              validation_data=val_generator, validation_steps=nb_val_steps,class_weight=class_weights, 
                              verbose=1)

In [ ]:
#saving model
import tensorflow as tf
model.save('vgg-esophagus-model.h5')

In [ ]:
from numpy import expand_dims
def tta_prediction(datagen, model, image, n_examples):
    # convert image into dataset
    samples = expand_dims(image, 0)
    # prepare iterator
    it = datagen.flow(samples, batch_size=n_examples)
    # make predictions for each augmented image
    probs = model.predict_generator(it, steps=n_examples, verbose=0)
    #print(len(probs))    
    prob = np.mean(probs, axis=1)    
    return prob

In [ ]:
from tqdm import tqdm
# configure image data augmentation
test_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

In [ ]:
test_imgs.shape

In [ ]:
targetvalues=[]
#i = 0
for img in tqdm(test_imgs):
    prediction=tta_prediction(test_datagen,model,img,32)
    pred_y = prediction[0]
    val=0
    if pred_y>0.5:
        val=1
    targetvalues.append(val)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test, targetvalues))
print(classification_report(y_test, targetvalues))

Manual Testing

In [ ]:
from PIL import Image
def preprocess_image(image_path, desired_size=128):
    im = Image.open(image_path)
    im = im.resize((desired_size, )*2, resample=Image.LANCZOS)
    return im

In [ ]:
import os
esotest = '/kaggle/input/esophagustestset/test/esophagus'
noesotest = '/kaggle/input/esophagustestset/test/no-esophagus'
esotestlist = os.listdir(esotest)
noesotestlist = os.listdir(noesotest)
testlist = []
for name in esotestlist:
    testlist.append(esotest+'/'+name)
for name in noesotestlist:
    testlist.append(noesotest+'/'+name)

In [ ]:
testlabels = []
for i in range(40):
    if i<20:
        testlabels.append(1)
    else:
        testlabels.append(0)

In [ ]:
N = 40
imSize = 128
# create an empty matrix for storing the images
testset = np.empty((40, imSize, imSize, 3), dtype=np.uint8)

# loop through the images from the images ids from the target\id dataset
# then grab the cooresponding image from disk, pre-process, and store in matrix in memory
for i, image_id in enumerate(tqdm(testlist)):
    testset[i, :, :, :] = preprocess_image(
        image_id
    )

In [ ]:
predval = []
for img in tqdm(testset):
    prediction=tta_prediction(test_datagen,model,img,1)
    pred_y = prediction[0]
    val=0
    if pred_y>0.5:
        val=1
    predval.append(val)

Testing the model accuracy manually

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
print(accuracy_score(testlabels,predval)*100)
print(confusion_matrix(testlabels, predval))